In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#데이터읽기
customers = pd.read_csv('./dacon_data/open/customers.csv')
locations = pd.read_csv('./dacon_data/open/locations.csv')
orders = pd.read_csv('./dacon_data/open/orders.csv')
order_items = pd.read_csv('./dacon_data/open/order_items.csv')
payments = pd.read_csv('./dacon_data/open/payments.csv')
products = pd.read_csv('./dacon_data/open/products.csv')
reviews = pd.read_csv('./dacon_data/open/reviews.csv')
sellers = pd.read_csv('./dacon_data/open/sellers.csv')
# 데이터 merge하기
temp = pd.merge(reviews,order_items, how='inner',on='Order_id')
temp = pd.merge(temp,orders, how='inner',on='Order_id')
temp = pd.merge(temp,payments, how='inner',on='Order_id')
temp = pd.merge(temp,products, how='inner',on='Product_id')
temp = pd.merge(temp,customers, how='inner',on='Customer_id')
temp = pd.merge(temp,sellers, how='inner',on='Seller_id')

In [2]:
temp.shape

(104956, 32)

In [82]:
# date 타입으로 변환
temp['Order_purchase_timestamp']=pd.to_datetime(temp['Order_purchase_timestamp'])
temp['Order_delivered_carrier_date']=pd.to_datetime(temp['Order_delivered_carrier_date'])
temp['Order_delivered_customer_date']=pd.to_datetime(temp['Order_delivered_customer_date'])
temp['Order_estimated_delivery_date']=pd.to_datetime(temp['Order_estimated_delivery_date'])
# temp[['Order_delivered_carrier_date','Order_delivered_customer_date',"Order_purchase_timestamp"]] = temp[['Order_delivered_carrier_date','Order_delivered_customer_date',"Order_purchase_timestamp"]].apply(pd.to_datetime)

In [83]:
temp['YearMonth'] = temp['Order_purchase_timestamp'].dt.strftime('%Y%m') #월별로 분류` 

In [84]:
temp = temp.drop(["Review_id","Order_id","Review_creation_date","Review_answer_timestamp","Order_item_id","Product_id","Seller_id"],axis=1)

In [85]:
temp = temp.drop(["Customer_id","Order_status","Order_purchase_timestamp","Order_delivered_carrier_date","Order_delivered_customer_date","Order_estimated_delivery_date","Payment_sequential"],axis=1)

In [86]:
temp = temp.drop(["Payment_installments","Payment_type","Product_weight_g","Product_length_cm","Product_height_cm","Product_width_cm","Customer_unique_id"],axis=1)

In [87]:
temp = temp.drop(["Seller_state","Seller_city","Seller_zipcode_prefix","Customer_state","Customer_city","Customer_zipcode_prefix"],axis=1)

In [88]:
temp.columns

Index(['Review_score', 'Price', 'Freight_value', 'Payment_value',
       'Product_category_name', 'YearMonth'],
      dtype='object')

In [89]:
temp.head(1)

,Review_score,Price,Freight_value,Payment_value,Product_category_name,YearMonth
0,4,185.0,13.63,397.26,sports_leisure,201901


In [90]:
temp_group_product_month = temp.groupby(["Product_category_name","YearMonth"])[["Price","Payment_value"]].sum().reset_index()

In [91]:
temp_group_product_month

,Product_category_name,YearMonth,Price,Payment_value
0,Unknown,201710,65.89,95.92
1,Unknown,201801,1603.64,1928.49
2,Unknown,201802,8517.92,10345.63
3,Unknown,201803,6920.95,8774.41
4,Unknown,201804,9923.90,11940.29
...,...,...,...,...
1191,watches_gifts,201903,98546.24,107352.18
1192,watches_gifts,201904,96108.64,106580.61
1193,watches_gifts,201905,120976.88,145705.65
1194,watches_gifts,201906,85985.37,105089.24


In [92]:
temp_group_product_month['Cumulative_Price'] = temp_group_product_month.groupby('Product_category_name')['Price'].cumsum()
temp_group_product_month['Cumulative_Payment_value'] = temp_group_product_month.groupby('Product_category_name')['Payment_value'].cumsum()

In [93]:
temp_group_product_month["Previous_Value"] = temp_group_product_month.groupby("Product_category_name")["Cumulative_Payment_value"].shift(1)

In [94]:
temp_group_product_month['Growth_Rate'] = ((temp_group_product_month['Cumulative_Payment_value'] - temp_group_product_month['Previous_Value']) / temp_group_product_month['Previous_Value']) * 100

In [95]:
temp_yearmonth_count = temp.groupby(["Product_category_name","YearMonth"])["Price"].count().reset_index()

In [96]:
temp_yearmonth_count = temp_yearmonth_count.rename(columns={'Price':'Count'})

In [97]:
temp_group_product_month= pd.merge(temp_group_product_month, temp_yearmonth_count, on=['Product_category_name','YearMonth'], how='inner')

In [98]:
temp_group_product_month.head()

,Product_category_name,YearMonth,Price,Payment_value,Cumulative_Price,Cumulative_Payment_value,Previous_Value,Growth_Rate,Count
0,Unknown,201710,65.89,95.92,65.89,95.92,NaN,NaN,2
1,Unknown,201801,1603.64,1928.49,1669.53,2024.41,95.92,2010.519183,12
2,Unknown,201802,8517.92,10345.63,10187.45,12370.04,2024.41,511.044205,56
3,Unknown,201803,6920.95,8774.41,17108.40,21144.45,12370.04,70.932754,62
4,Unknown,201804,9923.90,11940.29,27032.30,33084.74,21144.45,56.470090,79


In [102]:
temp_group_product_month['Cumulative_Count'] = temp_group_product_month.groupby('Product_category_name')['Count'].cumsum()
temp_group_product_month["Previous_Count"] = temp_group_product_month.groupby("Product_category_name")["Cumulative_Count"].shift(1)
temp_group_product_month['Growth_Rate_Count'] = ((temp_group_product_month['Cumulative_Count'] - temp_group_product_month['Previous_Count']) / temp_group_product_month['Previous_Count']) * 100

In [112]:
temp_group_product_month.head()

,Product_category_name,YearMonth,Growth_Rate,Growth_Rate_Count
0,Unknown,201710,NaN,NaN
1,Unknown,201801,2010.519183,600.000000
2,Unknown,201802,511.044205,400.000000
3,Unknown,201803,70.932754,88.571429
4,Unknown,201804,56.470090,59.848485


In [113]:
temp_group_product_month.columns

Index(['Product_category_name', 'YearMonth', 'Growth_Rate',
       'Growth_Rate_Count'],
      dtype='object')

In [116]:
# temp_group_product_month.info()

In [106]:
temp_group_product_month=temp_group_product_month.drop(["Price","Payment_value","Payment_value","Previous_Value","Count","Cumulative_Count","Previous_Count"],axis=1)

In [109]:
temp_group_product_month=temp_group_product_month.drop(["Cumulative_Price","Cumulative_Payment_value"],axis=1)

In [114]:
temp_group_product_month.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Growth_Rate,Growth_Rate_Count
Growth_Rate,1.000000,0.367110
Growth_Rate_Count,0.367110,1.000000


In [99]:
# 판매개수가 많아지면 이득이다 #0.36을 유의미한 값으로 잡아야하나

In [120]:
# temp_group_product_month

In [121]:
temp_group_product_month.Product_category_name.unique()

array(['Unknown', 'agro_industry_and_commerce', 'air_conditioning',
       'arts', 'arts_and_crafts', 'audio', 'automotive', 'baby',
       'bed_bath_table', 'bedroom_furniture', 'blu_ray_dvds',
       'books_general_interest', 'christmas_articles', 'cinema_photo',
       'computers_accessories', 'consoles_games', 'construction_tools',
       'construction_tools_construction', 'construction_tools_garden',
       'construction_tools_lighting', 'construction_tools_safety',
       'cool_stuff', 'diapers_hygiene', 'drinks', 'electronics',
       'fashion_bags_accessories', 'fashion_childrens_clothes',
       'fashion_mens_clothing', 'fashion_shoes', 'fashion_sport',
       'fashion_underwear_beachwear', 'fashion_womens_clothing',
       'flowers', 'food', 'food_drink', 'furniture_decor',
       'furniture_mattress_and_upholstery', 'garden_tools',
       'health_beauty', 'home_appliances', 'home_appliances_2',
       'home_comfort', 'home_comfort_2', 'home_construction',
       'home_utilit

In [124]:
temp_group_product_month_un = temp_group_product_month[temp_group_product_month.Product_category_name=="arts"]
temp_group_product_month_un

,Product_category_name,YearMonth,Growth_Rate,Growth_Rate_Count
58,arts,201803,NaN,NaN
59,arts,201804,57.713725,16.666667
60,arts,201805,1815.836587,85.714286
61,arts,201806,3.703478,46.153846
62,arts,201807,5.317718,15.789474
63,arts,201808,3.767719,18.181818
64,arts,201809,4.991279,11.538462
65,arts,201810,4.367872,10.344828
66,arts,201811,0.969637,6.250000
67,arts,201812,1.984824,11.764706


In [ ]:
# 어제 말한 물품준비시간을 50퍼를 줄이면 10퍼의 리뷰점수가 오르는 효과를 볼수있다를 제외하고는 딱히 유의미한 
# 결과가 보이진 않습니다만
# dataset info를 가지고 생각을 해봐도, kpi를 올리는 쪽으로 관계를 지어보면, 화물가격을 건들거나(하지만 이거는 안하기로 했음), 
# 배송날짜를 땡기는 쪽이 매출액을 올려준다는 근거가 있거나, 할부와 거래가치의 연관성이 설사 존재한다 하더라도 할부를 한다는게 
# 거래가치를 올려준다는 근거가 없고, 제품의 무게,길이등은 연관성이 없다고 판단되며, 리뷰점수와 판매액을 연관지으려고 해도 
# 물론 전처리 과정에서 어떻게 하냐에 따라서 상관계수가 다르게 나오겠지만 리뷰점수와 매출액이 양의 관계인가?는 잘 몰겠습니다.

In [ ]:
# Dataset Info.

# customers.csv [파일]
# 고객과 관련된 정보
# Customer_id : 고객 ID
# Customer_unique_id : 고객 고유 ID
# Customer_zipcode_prefix : 고객 우편번호 앞부분
# Customer_city : 고객 도시
# Customer_state : 고객 주


\


# order_items.csv [파일]
# 주문 아이템과 관련된 정보
# Order_id : 주문 고유 ID
# Order_item_id : 동일한 주문에 포함된 품목 수를 식별하는 일련 번호
# Product_id : 제품 고유 ID
# Seller_id : 판매자 고유 ID
# Price : 판매 가격
# Freight_value : 품목 화물 가격


# orders.csv [파일]
# 주문과 관련된 정보
# Order_id : 주문 고유 ID
# Customer_id : 고객 ID
# Order_status : 주문 상태
# Order_purchase_timestamp : 구매 시간
# Order_delivered_carrier_date : 물류 처리 시간
# Order_delivered_customer_date : 실제 배송 날짜
# Order_estimated_delivery_date : 기대 배송 날짜


# payments.csv [파일]
# 지불과 관련된 정보
# Order_id : 주문 고유 ID
# Payment_sequential : 결제 시퀀스(둘 이상의 결제 방법으로 결제 가능)
# Payment_type : 지불 방법 
# Payment_installments : 할부 횟수
# Payment_value : 거래 가치


# products.csv [파일]
# 제품과 관련된 정보
# Product_id : 제품 고유 ID
# Product_category_name : 카테고리 이름
# Product_weight_g : 제품 무게(g)
# Product_length_cm : 제품 길이(cm)
# Product_height_cm : 제품 높이(cm)
# Product_width_cm : 제품 너비(cm)


# reviews.csv[파일]
# Review_id : 리뷰 고유 ID
# Order_id : 주문 고유 ID
# Review_score : 리뷰 점수
# Review_creation_date : 리뷰 생성 시간
# Review_answer_timestamp : 리뷰 답변 시간


# sellers.csv[파일]
# Seller_id : 판매자 고유 ID
# Seller_zipcode_prefix : 판매자 우편번호 앞자리
# Seller_city : 판매자 도시(city)
# Seller_state : 판매자 주(state)